In [ ]:
#@title Install Dependencies { display-mode: "form" }

!git clone https://github.com/harrisonvanderbyl/rwkv_chatbot.git
!pip install inquirer transformers sty
%cd rwkv_chatbot
import os

In [6]:
#@title Choose Model and options { display-mode: "form" }
from src.rwkv import RWKV

model = '3B' #@param ["3B","7B","14B"]
links = {
    "3B": "https://huggingface.co/BlinkDL/rwkv-4-pile-3b/resolve/main/RWKV-4-Pile-3B-20221110-ctx4096.pth",
    "7B": "https://huggingface.co/BlinkDL/rwkv-4-pile-7b/resolve/main/RWKV-4-Pile-7B-20230109-ctx4096.pth",
    "14B": "https://huggingface.co/BlinkDL/rwkv-4-pile-14b/resolve/main/RWKV-4-Pile-14B-20230115-5775.pth"
}

selected = links[model]
fileName = selected.split("main/")[1]
if os.system(f"ls {fileName}"):
  os.system(f"wget {selected}")

backend = "pytorch(cpu/gpu)" #@param ["tensorflow(cpu/gpu)","pytorch(cpu/gpu)","jax(cpu/gpu/tpu)","pytorch-quant(gpu-8bit)"]

useGPU = True #@param {type:"boolean"}

matrix = 'bf16' #@param ["bf16","fp16","fp32"]
runtime = 'bf16' #@param ["bf16","fp32"]

kwargs = {}

if(backend == "pytorch(cpu/gpu)" or backend == "tensorflow(cpu/gpu)"):
  kwargs["useGPU"] = useGPU

if("pytorch" in backend):
  import torch 
  kwargs["runtimedtype"] = torch.float32 if "32" in runtime else torch.bfloat16 if "bf" in runtime else torch.float16
  if(backend == "pytorch(cpu/gpu)"):
    kwargs["dtype"] = torch.float32 if "32" in matrix else torch.bfloat16 if "bf" in matrix else torch.float16
  if(backend == "pytorch-quant(gpu-8bit)"):
    kwargs["chunksize"] = 4
  


model = RWKV(fileName,backend, **kwargs)


init RWKVOPS, from super


100%|██████████| 582/582 [00:02<00:00, 244.44it/s]


In [11]:
#@title Ask Question { display-mode: "form" }
model.resetState()
Prompt = "Who is Elon Musk" #@param {type:"string"}
model.loadContext("\n",
                                     f"Prompt: {Prompt}?\n\nLong Detailed Expert Response: ")
out = model.forward(number=100)["output"]
import IPython
IPython.display.HTML(f"<p>{out}</p>")

100%|██████████| 17/17 [00:00<00:00, 33.78it/s]
